# 使用 gensim 训练中文词向量 (word2vec)



word2vec算法通过将语词(word)映射到N维的向量空间，然后基于这个词向量可以进行聚类，找到近似词以及词性分析等相关的应用。

关于word2vec原理和核心算法CBOW（Continuous Bag-Of- Words），Skip-Gram在
[Word2vec 詞嵌入 (word embeddings) 的基本概念](https://github.com/erhwenkuo/deep-learning-with-keras-notebooks/blob/master/8.2-word2vec-concept-introduction.ipynb)已经进行了解释，不过对如何训练word2vec的模型并没有太多着墨。

这篇文章里将使用维基百科的中文语料，并使用python的gensim套件来训练word2vec的模型。


![gensim](http://x-wei.github.io/images/dlMOOC_L4/pasted_image001.png)

### 需求

- [Python 3.5](https://www.python.org/)
- [Anaconda](https://anaconda.org/)
- [gensim](https://radimrehurek.com/gensim/) : word2vec模型训练
- [jieba](https://github.com/fxsjy/jieba/) : 中文分词
- [hanziconv](https://github.com/berniey/hanziconv/) : 简繁转换

### 安裝

建议使用Windows的朋友使用Anandoa来设定相关的Python环境。

1. 安装Anaconda
2. 产生一个Anaconda环境
3. 安装库:

    
```
pip install gensim
pip install jieba
pip install hanziconv
```
### 下载训练用的相关脚本
1. 下載[Alex-CHUN-YU/Word2vec](https://github.com/Alex-CHUN-YU/Word2vec)的Github文件夹
```
git clone https://github.com/Alex-CHUN-YU/Word2vec.git
```

### 下载中文维基百科数据
1. 到 [中文维基百科dump](https://dumps.wikimedia.org/zhwiki/) 的目录下找到最新的dump资料档zhwiki-yyyymmdd-pages-articles.xml.bz2, 比如[zhwiki-20180220-pages-articles.xml.bz2 (1.4 GB)](https://dumps.wikimedia.org/zhwiki/20180220/zhwiki-20180220-pages-articles.xml.bz2)到'Word2vec/data'的子目录下

### 下载jieba字典
1. 以"Download ZIP"的方式下载 [fxsjy/jieba](https://github.com/fxsjy/jieba), 解压后將"extra_dict"整个目录复制到"Word2vec/model"子目录下

### 库的结构
   
你的目录结构看起来像这样: (这里只列出来在这个范例会用到的相关档案与目录)
```
Word2vec/
├──xxxx.ipynb (代表本例的notebook)
├── main.py
├── segmentation.py
├── train.py
├── wiki_to_txt.py
├── stopwords.txt
├── model/
│   └── extra_dict
│       ├── dict.txt.big
│       ├── dict.txt.small
│       ├── idf.txt.big
│       └── stop_words.txt
└── data/
    └── zhwiki-20180220-pages-articles.xml.bz2
```

## 训练流程
1. 取得中文维基数据，本次实验是采用 2018/02/20 的数据。
2. 将下载后的维基数据置于与"data/"子目录，再使用gensim.corpora的WikiCorpus函数来从wiki的xml档案中提取出维基文章的语词
3. 简体转繁体，再进行断词并同步过滤停用词
4. 训练并产生 word2vec 模型
5. 验证word2vec近似词以及词性分析等相关功能


### 载入相关库

In [2]:
!pip install  hanziconv 

In [3]:
# 把一些警告的讯息暂时关掉
import warnings
warnings.filterwarnings('ignore')

# Utilities相关库
import os
import numpy as np
import mmap
from tqdm import tqdm

# 可视化相关库
import jieba
from gensim.corpora import WikiCorpus
from gensim.models import word2vec
from hanziconv import HanziConv
import matplotlib.pyplot as plt

## 参数设定

In [4]:
# 文档的根目录路径
ROOT_DIR = os.getcwd()

# 训练/验证用的文档路径
DATA_PATH = os.path.join(ROOT_DIR, "data")

# 模型目录
MODEL_PATH = os.path.join(ROOT_DIR, "model")

# 设定jieba自定义字典路径
JIEBA_DICTFILE_PATH = os.path.join(MODEL_PATH,"extra_dict", "dict.txt.big")

# 设定字典
jieba.set_dictionary(JIEBA_DICTFILE_PATH)

## 歩驟 1. 取得语料 (Corpus)

由于 word2vec 是基于非监督式学习，语料涵盖的越全面，训练出来的结果也会越漂亮。在本文中所采用的是维基百科于2018/02/20的dump档，文章篇数共有 309602 篇。因为维基百科会定期更新备份数据，如果 8 月 20 号的备份不幸地被删除了，也可以前往维基百科:数据库下载挑选更近期的数据，不过请特别注意一点，我们要挑选的是以 pages-articles.xml.bz2 结尾的备份，而不是以 pages-articles-multistream.xml.bz2 结尾的备份，否则会在清理上出现一些异常，无法正常解析文章。

初始化WikiCorpus后，能藉由get_texts()可迭代每一篇wikipedia的文章，它所回传的是一个tokens list，我们以空白符将这些 tokens 串接起来，统一输出到同一份文本文件里。这边要注意一件事，get_texts()受wikicorpus.py中的变量ARTICLE_MIN_WORDS限制，只会回传内容长度大于 50 的文章。


In [3]:
# 将wiki数据集下载后进行提取，且将 xml 转换成plain txt 
wiki_articles_xml_file = os.path.join(DATA_PATH, "zhwiki-20180220-pages-articles.xml.bz2")
wiki_articles_txt_file = os.path.join(DATA_PATH, "zhwiki_plaintext.txt")

# 使用gensim.WikiCorpus来读取wiki XML中的corpus
wiki_corpus = WikiCorpus(wiki_articles_xml_file, dictionary = {})

# 迭代提取出來的词汇
with open(wiki_articles_txt_file, 'w', encoding='utf-8') as output:
    text_count = 0
    for text in wiki_corpus.get_texts():
        # 把词汇写进文件中备用
        output.write(' '.join(text) + '\n')
        text_count += 1
        if text_count % 10000 == 0:
            print("目前已处理 %d 篇文章" % text_count)

print("简繁转换已完毕, 总共处理了 %d 篇文章!"% text_count)

在4 cores(AMD) 8 GB内存的计算机上, 以上歩骤花了将近20分钟。

## 歩驟 2. 进行中文分词与stop-word移除

我们有清完XML标签的语料了，再来就是要把语料中每个句子，进一步拆解成语词，这个步骤称为「分词」。中文分词的工具有很多，这里采用的是jieba。在wiki的中文文档中有简体跟繁体混在一起的情形，所以我们在分词前，还需加上一道简繁转换的手续。


In [5]:
# 一个取得一个文件行数的函数式
def get_num_lines(file_path):
    fp = open(file_path, "r+")
    buf = mmap.mmap(fp.fileno(), 0)
    lines = 0
    while buf.readline():
        lines += 1
    return lines

In [6]:
# 進行簡體轉繁體
wiki_articles_zh_tw_file = os.path.join(DATA_PATH, "zhwiki_zh_tw.txt")

wiki_articles_zh_tw = open(wiki_articles_zh_tw_file, "w", encoding = "utf-8")

# 迭代转换成plain text的wiki文件, 并透过HanziConv来进行简繁转换
with open(wiki_articles_txt_file, "r", encoding = "utf-8") as wiki_articles_txt:
    for line in tqdm(wiki_articles_txt, total=get_num_lines(wiki_articles_txt_file)):
        wiki_articles_zh_tw.write(HanziConv.toTraditional(line))
        
print("成功简繁转换!")

wiki_articles_zh_tw.close()

In [7]:
# 进行中文分词并同步停用词过滤
stops_word_file = os.path.join(ROOT_DIR, "stopwords.txt")

# stopword字词集
stopwordset = set()

# 读取stopword词典，并保存到stopwordset中
with open("stopwords.txt", "r", encoding = "utf-8") as stopwords:
    for stopword in stopwords:
        stopwordset.add(stopword.strip('\n'))

# 保存分词后的结果
wiki_articles_segmented_file = os.path.join(DATA_PATH, "zhwiki_segmented.txt")
wiki_articles_segmented = open(wiki_articles_segmented_file, "w", encoding = "utf-8")

# 迭代转换成繁体的wiki文檔, 并透过jieba来进行分词
with open(wiki_articles_zh_tw_file, "r", encoding = "utf-8") as Corpus:
    for sentence in tqdm(Corpus, total=get_num_lines(wiki_articles_zh_tw_file)):
    #for sentence in Corpus:
        sentence = sentence.strip("\n")
        pos = jieba.cut(sentence, cut_all = False)
        for term in pos:
            if term not in stopwordset:
                wiki_articles_segmented.write(term + " ")
                
print("jieba分词完毕，并已完成过滤词工序!")
wiki_articles_zh_tw_file.close()

### 停用词与Word2Vec

停用词(stop word)就是像英文中的 the,a,this，中文的你我他，与其他词相比显得不怎么重要，对文章主题也无关紧要的，就可以将它视为停用词。而要排除停用词的理由，其实与word2vec的实作有着相当大的关系。

在word2vec有一个概念叫"窗口( windows )"。

很显然，一个词的意涵跟他的左右邻居很有关系，比如「雨越下越大，茶越充越淡」，什么会「下」？「雨」会下，什么会「淡」？茶会「淡」，这样的模拟举不胜举，那么，若把思维逆转过来呢？

显然，我们或多或少能从左右邻居是谁，猜出中间的是什么，这很像我们国高中时天天在练的英文克漏字。那么问题来了，左右邻居有谁？能更精确地说，你要往左往右看几个？假设我们以「孔乙己 一到 店 所有 喝酒 的 人 便都 看着 他 笑」为例，如果往左往右各看一个：

>1   [孔乙己 一到] 店 所有 喝酒 的 人 便 都 看著 他 笑

>2   [孔乙己 一到 店] 所有 喝酒 的 人 便 都 看著 他 笑

>3   孔乙己 [一到 店 所有] 喝酒 的 人 便 都 看著 他 笑

>4   孔乙己 一到 [店 所有 喝酒] 的 人 便 都 看著 他 笑

>5   ......

这样就构成了一个 size=1 的 windows，这个 1 是极端的例子，为了让我们看看有停用词跟没停用词差在哪，这句话去除了停用词应该会变成：

>1   孔乙己 一到 店 所有 喝酒 人 看着 笑

我们看看「人」的窗口变化，原本是「的 人 便」，后来是「喝酒 人 看着」，相比原本的情形，去除停用词后，我们对「人」这个词有更多认识，比如人会喝酒，人会看东西，当然啦，这是我以口语的表达，机器并不会这么想，机器知道的是人跟喝酒会有某种关联，跟看会有某种关联，但尽管如此，也远比本来的「的 人 便」好太多太多了。


## 歩骤 3. 训练词向量

这是最简单的部分，同时也是最困难的部分，简单的是程序代码，困难的是词向量效能上的微调与后训练。

相关参数:
* sentences:这是要训练的句子集
* size:这表示的是训练出的词向量会有几维
* alpha:机器学习中的学习率，这东西会逐渐收敛到 min_alpha
* sg:sg=1表示采用skip-gram,sg=0 表示采用cbow
* window:还记得孔乙己的例子吗？能往左往右看几个字的意思
* workers:线程数目，建议别超过 4
* min_count:若这个词出现的次数小于min_count，那他就不会被视为训练对象


In [1]:
from gensim.models import word2vec

# 可参考 https://radimrehurek.com/gensim/models/word2vec.html 更多运用
print("word2vec模型训练中...")

#加载文件
sentence = word2vec.Text8Corpus(wiki_articles_segmented_file)

# 设置参数和训练模型(Train)
model = word2vec.Word2Vec(sentence, size = 300, window = 10, min_count = 5, workers = 4, sg = 1)

# 保存模型
word2vec_model_file = os.path.join(MODEL_PATH, "zhwiki_word2vec.model")

model.wv.save_word2vec_format(word2vec_model_file, binary = True)

#model.wv.save_word2vec_format("wiki300.model.bin", binary = True)
print("Word2vec模型已存储完毕")

## 词向量实验

训练完成后，让我们来测试一下模型的效果。由于 gensim 会将整个模型读了进来，所以内存会消耗相当多。


In [12]:
from gensim.models.keyedvectors import KeyedVectors

word_vectors = KeyedVectors.load_word2vec_format(word2vec_model_file, binary = True)

In [13]:
print("词汇相似词前 5 排序")
query_list=['校长']
res = word_vectors.most_similar(query_list[0], topn = 5)
for item in res:
    print(item[0] + "," + str(item[1]))

In [14]:
print("计算2个词汇间的 Cosine 相似度")
query_list=['爸爸','妈妈']
res = word_vectors.similarity(query_list[0], query_list[1])
print(res)

In [15]:
query_list=['爸爸','老公','妈妈']
print("%s之于%s，如%s之于" % (query_list[0], query_list[1], query_list[2]))
res = word_vectors.most_similar(positive = [query_list[0], query_list[1]], negative = [query_list[2]], topn = 5)
for item in res:
    print(item[0] + "," + str(item[1]))

### 参考资料:
* [以 gensim 训练中文词向量](http://zake7749.github.io/2016/08/28/word2vec-with-gensim/)
* [Alex-CHUN-YU/Word2vec](https://github.com/Alex-CHUN-YU/Word2vec/blob/master/train.py)